In [1]:
import os
import pandas as pd
import math
import numpy as np
import json

from datetime import datetime
from pathlib import Path

In [2]:
#get current os directory
cwd = os.getcwd()
current_dir = Path(cwd).parents[1]
print(current_dir)

/Users/berenicegudino/Desktop/Code/dpp


In [ ]:
company = 'OnePan'  
path = os.path.join(current_dir,Path('dpp_app/data'),company)
configuration_file = 'parameter_metadata_'+company+'.xlsx'
destination_path = os.path.join(current_dir,Path('dpp_api/data'),company)

In [4]:
#Read the configuration file
configuration_file_df = pd.read_excel(os.path.join(path,configuration_file))
configuration_file_df.head()

,category,Relevant for use case,sub_category,parameter,order_parameter,subparameter,access,type,example data(text),example data,data model definition string
0,1. General product and manufacturer information,x,1. General product and manufacturer information,DPP Created Timestamp,1,NaN,Public,Static,0,NaN,"{""DPP Created Timestamp"" : {""value"":{""@value"":..."
1,1. General product and manufacturer information,x,1. General product and manufacturer information,Product ID,2,NaN,Public,Static,0,1,"{""Product ID"": {""@type"":""gs1:gtin"",""value"": {""..."
2,1. General product and manufacturer information,x,1. General product and manufacturer information,Data carrier type,3,NaN,Public,Static,0,NaN,"{""Data carrier type"" : {""value"":{""@value"":"""", ..."
3,1. General product and manufacturer information,x,1. General product and manufacturer information,Manufacturing date,4,NaN,Public,Static,0,NaN,"{""Manufacturing date"": {""@type"":""gs1:productio..."
4,1. General product and manufacturer information,x,1. General product and manufacturer information,Manufactured by,5,NaN,Public,Static,0,NaN,"{""Manufactured by"": {""@type"":""gs1:organization..."


In [5]:
#Sanity check on data model definition column
def incorrect_json(x):
    try:
        json.loads(x)
        return False
    except:
        return True

filter_ij = configuration_file_df['data model definition string'].apply(lambda x: incorrect_json(x))
print('Json format in Data Model Definition column is incorrect in:')
print(configuration_file_df.loc[filter_ij,['data model definition string']])

Json format in Data Model Definition column is incorrect in:
Empty DataFrame
Columns: [data model definition string]
Index: []


In [6]:
def incorrect_parameter_name(dmd, parameter):
    try:
        json.loads(dmd)[parameter]
        return True
    except:
        return False  

filter_ipn = configuration_file_df.apply(lambda x: incorrect_parameter_name(x['data model definition string'],x['parameter']), axis=1)
print('Name parameter Data Model Definition column is incorrect in:')
print(configuration_file_df.loc[np.logical_not(filter_ipn),'parameter'])

Name parameter Data Model Definition column is incorrect in:
Series([], Name: parameter, dtype: object)


In [7]:
#Create dataframe with static parameters
source_static_parameters_df = configuration_file_df[configuration_file_df['type'] == 'Static']
source_static_parameters_df.reset_index(drop=True,inplace=True)
source_static_parameters_df.head()

,category,Relevant for use case,sub_category,parameter,order_parameter,subparameter,access,type,example data(text),example data,data model definition string
0,1. General product and manufacturer information,x,1. General product and manufacturer information,DPP Created Timestamp,1,NaN,Public,Static,0,NaN,"{""DPP Created Timestamp"" : {""value"":{""@value"":..."
1,1. General product and manufacturer information,x,1. General product and manufacturer information,Product ID,2,NaN,Public,Static,0,1,"{""Product ID"": {""@type"":""gs1:gtin"",""value"": {""..."
2,1. General product and manufacturer information,x,1. General product and manufacturer information,Data carrier type,3,NaN,Public,Static,0,NaN,"{""Data carrier type"" : {""value"":{""@value"":"""", ..."
3,1. General product and manufacturer information,x,1. General product and manufacturer information,Manufacturing date,4,NaN,Public,Static,0,NaN,"{""Manufacturing date"": {""@type"":""gs1:productio..."
4,1. General product and manufacturer information,x,1. General product and manufacturer information,Manufactured by,5,NaN,Public,Static,0,NaN,"{""Manufactured by"": {""@type"":""gs1:organization..."


In [8]:
source_dynamic_parameters_df = configuration_file_df[np.logical_or(configuration_file_df['type'] == 'Dynamic', configuration_file_df['parameter'] == 'Product ID')]
source_dynamic_parameters_df.reset_index(drop=True,inplace=True)
source_dynamic_parameters_df.head()

,category,Relevant for use case,sub_category,parameter,order_parameter,subparameter,access,type,example data(text),example data,data model definition string
0,1. General product and manufacturer information,x,1. General product and manufacturer information,Product ID,2,NaN,Public,Static,0,1,"{""Product ID"": {""@type"":""gs1:gtin"",""value"": {""..."
1,9. Product Updates,x,9. Product Updates,Update ID,18,NaN,public,Dynamic,1.1,1.1,"{""Update ID"" : { ""value"": {""@value"":"""", ""@type..."
2,9. Product Updates,x,9. Product Updates,DPP Update Timestamp,19,NaN,Public,Dynamic,0,NaN,"{""DPP Update Timestamp"" : {""@type"":""schema:sta..."
3,9. Product Updates,x,9. Product Updates,DPP Update Reason,20,NaN,Public,Dynamic,0,NaN,"{""DPP Update Reason"" : {""@type"":""schema:descr..."
4,9. Product Updates,x,9. Product Updates,DPP Update Responsible,21,NaN,Public,Dynamic,Brighteco,NaN,"{""DPP Update Responsible"" : {""@type"":""schema:..."


In [9]:
def get_value(parameter, dmd, value):
    if value == 'File': #Create an empty text file
        valid_file_name = "".join(x for x in parameter if x.isalnum())
        path_file = os.path.join(destination_path, valid_file_name + '.txt')
        with open(path_file, 'w') as f:
            f.write('')
        return path_file
    elif not(type(value)==float and math.isnan(value)): #If the value is given
        return value
    elif 'xsd:string' in dmd or 'rdf:langString' in dmd: #Otherwise return a default value
        return '-'
    elif 'xsd:integer' in dmd:
        return 0
    elif 'xsd:float' in dmd:
        return 0.0
    elif 'xsd:boolean' in dmd:
        return 0
    elif 'xsd:date' in dmd:
        return datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    elif 'xsd:int' in dmd:
        return 0

def strip_values(values_str):
    #Get subparameters    
    values_tmp = values_str[1:-1].split(',')
    #Remove blank spaces
    values = [x.strip() for x in values_tmp]
    return values

def create_database_file(source_df, path_db_name):
    #Create new dataframe taken parameters as columns and example data as values
    parameters_df = pd.DataFrame()
    for index, row in source_df.iterrows():
        if type(row['subparameter'])==float and math.isnan(row['subparameter']):
            parameters_df.loc[0,row['parameter']] = get_value(row['parameter'], row['data model definition string'], row['example data'])
        else:   
            subparameters = strip_values(row['subparameter'])
            if not(type(row['example data'])==float and math.isnan(row['example data'])): #Get values if there is any
                raw_data = strip_values(row['example data'])
            else: #Create dummy data
                raw_data = []
                for subparam in subparameters:
                    raw_data.append(get_value(row['parameter'], str(json.loads(row['data model definition string'])[row['parameter']][subparam]), row['example data']))
            for value, subparameter in zip(raw_data,subparameters):
                parameters_df.loc[0,row['parameter']+'.'+subparameter] = value     
    #Save the dataframe to a csv file
    parameters_df.to_csv(path_db_name, index=False)

In [10]:
#Create static database file
create_database_file(source_static_parameters_df, os.path.join(destination_path,'static.csv'))
#Create dynamic database file
create_database_file(source_dynamic_parameters_df, os.path.join(destination_path,'dynamic.csv'))